In [1]:
# create for run task 2.3

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gzip
import random
import time
import os

In [4]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
f = 'drive/Shareddrives/EECS522/mnist.pkl.gz'

In [6]:

def load_data(f):
    
    '''
    load the MNIST data as a tuple contraining the training dta, evaluation data, and the test data.
    Training data has two entries, one is actual training images, which is a numpy ndarray with 50,000 entries, each entry is a numpy ndarray with 784 (28*28) pixels in a single MNIST image
                                  the other is numpy ndarray containing 50,000 entries. Those entries are just the digit values (0...9) for the corresponding images contained in the first entry of the tuple. 
    The ``validation_data`` and ``test_data`` are similar, except each contains only 10,000 images.
    
     This is a nice data format, but for use in neural networks it's helpful to modify the format of the ``training_data`` a little. That's done in the wrapper function ``load_data_wrapper()``, see
below.
    '''
    
    f = gzip.open(f, 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    
    return (training_data, validation_data, test_data)

def load_data_wrapper(f):
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data(f)
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y,10) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    
    return (training_data, validation_data, test_data)


def vectorized_result(j,n):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((n, 1))
    e[j] = 1.0
    return e

In [7]:
class Network(object):
    
    def __init__(self, sizes):
        
        
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        # initialize weights and biases
        # by different scenairos
        # yangsong gu 10/30/2021, 5:36PM

        if self.sizes[0] > 1000:
            self.large_weight_initialization()
        else:
            self.small_weight_initialization()

        # for momentum based gradient descent
        self.weights_vel = [np.zeros((y, x))
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]
        self.biases_vel = [np.zeros((y, 1)) for y in self.sizes[1:]]
        
  
    def large_weight_initialization(self):

        # from nielson book, which is basic initilization approach, 
        # the weights is not standardized. 
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def small_weight_initialization(self):

        # standardized the weights, using sqrt(number of inputs)
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a):
            
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):

            a = self.sigmoid(np.dot(w, a)+b)

        return a
        
    def SGD(self, training_data, epochs, mini_batch_size, eta, subtype, test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n_train = len(training_data)


        loss_Tr = [] # loss of training set  use upper case T to denote a set of samples
        loss_Te = [] # loss of test set
        acc_Tr  = [] # accuracy of training set
        acc_Te  = [] # accuracy of test set
    
        f = open('drive/Shareddrives/EECS522/newvision/{}_model_{}ep{}_mb{}_eta{}.txt'.format(subtype,self.sizes, epochs,mini_batch_size, eta), 'w')

        f.write('Layers {} learning rate: {}, min_batch: {}, epochs:{} \n'.format(self.sizes, 
                eta, mini_batch_size,epochs))
      

        f.write('epoch, loss_tr,loss_te, acc_tr, acc_te \n')
        
        if test_data:

            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n_train, mini_batch_size)]
            
            if  'mom' in subtype:
                for mini_batch in mini_batches:
                    self.update_mini_batch_momentum(mini_batch, eta)
            else:
                for mini_batch in mini_batches:
                    self.update_mini_batch(mini_batch, eta)
                
            if test_data:
                print("Epoch {}: {} / {}".format(j, self.evaluate(test_data), n_test))
                          
            else:
                print("Epoch {} complete".format(j))
            
            # compute the loss for training data set
            loss_tr = [0.5 * np.sum((self.feedforward(x) - y)**2) for (x, y) in training_data]

            loss_Tr.append(sum(loss_tr))
            
            
        
            # write the accuracy to 
            # compute the loss for test data set
            if test_data:
                loss_te = [0.5 * np.sum((self.feedforward(x) - vectorized_result(y,self.sizes[-1]))**2) for (x, y ) in test_data]
                loss_Te.append(sum(loss_te))
            # compute the accuracy for training data

            res_tr = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x,y) in training_data]
            # acc_tr = self.evaluate(training_data) / n_train
            acc_tr = sum(int(x == y) for (x, y) in res_tr) / n_train

            acc_Tr.append(acc_tr)
            
            # accuracy of test set
            if test_data:

                acc_te = self.evaluate(test_data)/n_test
                acc_Te.append(acc_te)
            
            f.write("{}, {:10}, {:10}, {:.5}, {:.5}\n".format(j, sum(loss_tr), sum(loss_te),acc_tr, acc_te))
            
        f.close()
        return loss_Tr, loss_Te, acc_Tr, acc_Te
    
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def update_mini_batch_momentum(self, mini_batch,eta):

      # apply momentum to update weights and biases
      # ys g 6:20PM 10/30/2021
      nabla_b = [np.zeros(b.shape) for b in self.biases]
      nabla_w = [np.zeros(w.shape) for w in self.weights]
      for x, y in mini_batch:
          delta_nabla_b, delta_nabla_w = self.backprop(x, y)
          nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
          nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

      mu = 0.9 # momentum
      self.weights_vel = [mu * v - (eta/len(mini_batch)) * nw for v, nw in zip(self.weights_vel, nabla_w)]
      self.biases_vel = [mu * v - (eta/len(mini_batch)) * nw for v, nw in zip(self.biases_vel, nabla_b)]

      self.weights  = [w  + v for w, v in zip(self.weights, self.weights_vel)]
      self.biases = [b + v for b, v in zip(self.biases, self.biases_vel)]

    
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = self.sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            self.sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = self.sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    
    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""

        # modeified by Yangsong 
        # y --> np.argmax(y) if there are multiple maximum
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        
        return sum(int(x == y) for (x, y) in test_results)
    
    
    
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)
    
    def sigmoid(self, z):
        '''simoid function'''
        return 1/(1+np.exp(-z))

    def sigmoid_prime(self, z):
        '''
        derivative of sigmoid function'''

        return self.sigmoid(z) * (1- self.sigmoid(z))

In [8]:
## TASK 2.3 use momentum 

In [12]:
# adjust network structure 0
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_momentum'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 30,30,30, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)


Epoch 0: 9029 / 10000
Epoch 1: 9344 / 10000
Epoch 2: 9446 / 10000
Epoch 3: 9471 / 10000
Epoch 4: 9509 / 10000
Epoch 5: 9470 / 10000
Epoch 6: 9503 / 10000
Epoch 7: 9495 / 10000
Epoch 8: 9589 / 10000
Epoch 9: 9586 / 10000
Epoch 10: 9500 / 10000
Epoch 11: 9527 / 10000
Epoch 12: 9560 / 10000
Epoch 13: 9556 / 10000
Epoch 14: 9592 / 10000
Epoch 15: 9583 / 10000
Epoch 16: 9603 / 10000
Epoch 17: 9590 / 10000
Epoch 18: 9575 / 10000
Epoch 19: 9560 / 10000
Epoch 20: 9574 / 10000
Epoch 21: 9587 / 10000
Epoch 22: 9580 / 10000
Epoch 23: 9572 / 10000
Epoch 24: 9560 / 10000
Epoch 25: 9611 / 10000
Epoch 26: 9595 / 10000
Epoch 27: 9603 / 10000
Epoch 28: 9617 / 10000
Epoch 29: 9614 / 10000
Epoch 30: 9549 / 10000
Epoch 31: 9587 / 10000
Epoch 32: 9566 / 10000
Epoch 33: 9564 / 10000
Epoch 34: 9565 / 10000
Epoch 35: 9604 / 10000
Epoch 36: 9624 / 10000
Epoch 37: 9586 / 10000
Epoch 38: 9606 / 10000
Epoch 39: 9618 / 10000
Epoch 40: 9599 / 10000
Epoch 41: 9595 / 10000
Epoch 42: 9592 / 10000
Epoch 43: 9610 / 1000

In [13]:
# adjust network structure 1
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_momentum'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 64,128,256, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 7935 / 10000
Epoch 1: 9386 / 10000
Epoch 2: 9494 / 10000
Epoch 3: 9610 / 10000
Epoch 4: 9555 / 10000
Epoch 5: 9590 / 10000
Epoch 6: 9611 / 10000
Epoch 7: 9645 / 10000
Epoch 8: 9661 / 10000
Epoch 9: 9690 / 10000
Epoch 10: 9704 / 10000
Epoch 11: 9695 / 10000
Epoch 12: 9703 / 10000
Epoch 13: 9654 / 10000
Epoch 14: 9698 / 10000
Epoch 15: 9697 / 10000
Epoch 16: 9685 / 10000
Epoch 17: 9691 / 10000
Epoch 18: 9717 / 10000
Epoch 19: 9702 / 10000
Epoch 20: 9682 / 10000
Epoch 21: 9718 / 10000
Epoch 22: 9701 / 10000
Epoch 23: 9724 / 10000
Epoch 24: 9707 / 10000
Epoch 25: 9730 / 10000
Epoch 26: 9738 / 10000
Epoch 27: 9704 / 10000
Epoch 28: 9729 / 10000
Epoch 29: 9745 / 10000
Epoch 30: 9726 / 10000
Epoch 31: 9717 / 10000
Epoch 32: 9739 / 10000
Epoch 33: 9712 / 10000
Epoch 34: 9725 / 10000
Epoch 35: 9733 / 10000
Epoch 36: 9732 / 10000
Epoch 37: 9709 / 10000
Epoch 38: 9737 / 10000
Epoch 39: 9735 / 10000
Epoch 40: 9746 / 10000
Epoch 41: 9735 / 10000
Epoch 42: 9724 / 10000
Epoch 43: 9729 / 1000

In [14]:
# adjust network structure 2
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_momentum'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 30, 30, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 9321 / 10000
Epoch 1: 9469 / 10000
Epoch 2: 9505 / 10000
Epoch 3: 9515 / 10000
Epoch 4: 9556 / 10000
Epoch 5: 9571 / 10000
Epoch 6: 9569 / 10000
Epoch 7: 9577 / 10000
Epoch 8: 9575 / 10000
Epoch 9: 9587 / 10000
Epoch 10: 9578 / 10000
Epoch 11: 9590 / 10000
Epoch 12: 9595 / 10000
Epoch 13: 9596 / 10000
Epoch 14: 9603 / 10000
Epoch 15: 9605 / 10000
Epoch 16: 9602 / 10000
Epoch 17: 9601 / 10000
Epoch 18: 9618 / 10000
Epoch 19: 9600 / 10000
Epoch 20: 9606 / 10000
Epoch 21: 9608 / 10000
Epoch 22: 9597 / 10000
Epoch 23: 9609 / 10000
Epoch 24: 9604 / 10000
Epoch 25: 9606 / 10000
Epoch 26: 9624 / 10000
Epoch 27: 9609 / 10000
Epoch 28: 9608 / 10000
Epoch 29: 9612 / 10000
Epoch 30: 9599 / 10000
Epoch 31: 9581 / 10000
Epoch 32: 9622 / 10000
Epoch 33: 9591 / 10000
Epoch 34: 9601 / 10000
Epoch 35: 9605 / 10000
Epoch 36: 9623 / 10000
Epoch 37: 9589 / 10000
Epoch 38: 9606 / 10000
Epoch 39: 9584 / 10000
Epoch 40: 9597 / 10000
Epoch 41: 9583 / 10000
Epoch 42: 9588 / 10000
Epoch 43: 9586 / 1000

In [15]:
# adjust network structure 3
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_momentum'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 60,60,60, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)
print(time.time() - t1)

Epoch 0: 9145 / 10000
Epoch 1: 9431 / 10000
Epoch 2: 9423 / 10000
Epoch 3: 9550 / 10000
Epoch 4: 9634 / 10000
Epoch 5: 9660 / 10000
Epoch 6: 9606 / 10000
Epoch 7: 9582 / 10000
Epoch 8: 9677 / 10000
Epoch 9: 9705 / 10000
Epoch 10: 9683 / 10000
Epoch 11: 9697 / 10000
Epoch 12: 9663 / 10000
Epoch 13: 9694 / 10000
Epoch 14: 9692 / 10000
Epoch 15: 9645 / 10000
Epoch 16: 9721 / 10000
Epoch 17: 9709 / 10000
Epoch 18: 9711 / 10000
Epoch 19: 9669 / 10000
Epoch 20: 9704 / 10000
Epoch 21: 9663 / 10000
Epoch 22: 9696 / 10000
Epoch 23: 9726 / 10000
Epoch 24: 9715 / 10000
Epoch 25: 9682 / 10000
Epoch 26: 9725 / 10000
Epoch 27: 9716 / 10000
Epoch 28: 9720 / 10000
Epoch 29: 9681 / 10000
Epoch 30: 9722 / 10000
Epoch 31: 9719 / 10000
Epoch 32: 9727 / 10000
Epoch 33: 9730 / 10000
Epoch 34: 9709 / 10000
Epoch 35: 9724 / 10000
Epoch 36: 9716 / 10000
Epoch 37: 9682 / 10000
Epoch 38: 9748 / 10000
Epoch 39: 9690 / 10000
Epoch 40: 9719 / 10000
Epoch 41: 9740 / 10000
Epoch 42: 9740 / 10000
Epoch 43: 9735 / 1000

In [16]:
# adjust network structure 4
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_momentum'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 60,60, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 9327 / 10000
Epoch 1: 9362 / 10000
Epoch 2: 9548 / 10000
Epoch 3: 9590 / 10000
Epoch 4: 9649 / 10000
Epoch 5: 9674 / 10000
Epoch 6: 9675 / 10000
Epoch 7: 9727 / 10000
Epoch 8: 9654 / 10000
Epoch 9: 9711 / 10000
Epoch 10: 9708 / 10000
Epoch 11: 9692 / 10000
Epoch 12: 9719 / 10000
Epoch 13: 9697 / 10000
Epoch 14: 9716 / 10000
Epoch 15: 9723 / 10000
Epoch 16: 9719 / 10000
Epoch 17: 9724 / 10000
Epoch 18: 9726 / 10000
Epoch 19: 9729 / 10000
Epoch 20: 9723 / 10000
Epoch 21: 9728 / 10000
Epoch 22: 9709 / 10000
Epoch 23: 9735 / 10000
Epoch 24: 9727 / 10000
Epoch 25: 9710 / 10000
Epoch 26: 9724 / 10000
Epoch 27: 9733 / 10000
Epoch 28: 9727 / 10000
Epoch 29: 9735 / 10000
Epoch 30: 9731 / 10000
Epoch 31: 9741 / 10000
Epoch 32: 9731 / 10000
Epoch 33: 9733 / 10000
Epoch 34: 9733 / 10000
Epoch 35: 9735 / 10000
Epoch 36: 9733 / 10000
Epoch 37: 9730 / 10000
Epoch 38: 9734 / 10000
Epoch 39: 9741 / 10000
Epoch 40: 9741 / 10000
Epoch 41: 9741 / 10000
Epoch 42: 9744 / 10000
Epoch 43: 9739 / 1000

In [7]:
# adjust network structure 5
model = {'min_batch':15, 'epoches':100, 'learningrate':1, 'comment': 'image_momentum'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 256,128,64, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 8587 / 10000
Epoch 1: 9450 / 10000
Epoch 2: 9455 / 10000
Epoch 3: 9609 / 10000
Epoch 4: 9654 / 10000
Epoch 5: 9651 / 10000
Epoch 6: 9731 / 10000
Epoch 7: 9681 / 10000
Epoch 8: 9728 / 10000
Epoch 9: 9745 / 10000
Epoch 10: 9759 / 10000
Epoch 11: 9778 / 10000
Epoch 12: 9796 / 10000
Epoch 13: 9800 / 10000
Epoch 14: 9770 / 10000
Epoch 15: 9795 / 10000
Epoch 16: 9784 / 10000
Epoch 17: 9777 / 10000
Epoch 18: 9793 / 10000
Epoch 19: 9804 / 10000
Epoch 20: 9809 / 10000
Epoch 21: 9786 / 10000
Epoch 22: 9797 / 10000
Epoch 23: 9803 / 10000
Epoch 24: 9786 / 10000
Epoch 25: 9812 / 10000
Epoch 26: 9808 / 10000
Epoch 27: 9823 / 10000
Epoch 28: 9823 / 10000
Epoch 29: 9803 / 10000
Epoch 30: 9807 / 10000
Epoch 31: 9803 / 10000
Epoch 32: 9813 / 10000
Epoch 33: 9802 / 10000
Epoch 34: 9805 / 10000
Epoch 35: 9805 / 10000
Epoch 36: 9816 / 10000
Epoch 37: 9816 / 10000
Epoch 38: 9801 / 10000
Epoch 39: 9811 / 10000
Epoch 40: 9817 / 10000
Epoch 41: 9819 / 10000
Epoch 42: 9819 / 10000
Epoch 43: 9815 / 1000

In [17]:
# adjust network structure 5
model = {'min_batch':20, 'epoches':50, 'learningrate':3, 'comment': 'image_momentum'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 256,128,64, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 9276 / 10000
Epoch 1: 9526 / 10000
Epoch 2: 9622 / 10000
Epoch 3: 9659 / 10000
Epoch 4: 9639 / 10000
Epoch 5: 9697 / 10000
Epoch 6: 9700 / 10000
Epoch 7: 9715 / 10000
Epoch 8: 9683 / 10000
Epoch 9: 9757 / 10000
Epoch 10: 9755 / 10000
Epoch 11: 9747 / 10000
Epoch 12: 9762 / 10000
Epoch 13: 9757 / 10000
Epoch 14: 9765 / 10000
Epoch 15: 9785 / 10000
Epoch 16: 9789 / 10000
Epoch 17: 9782 / 10000
Epoch 18: 9772 / 10000
Epoch 19: 9784 / 10000
Epoch 20: 9785 / 10000
Epoch 21: 9795 / 10000
Epoch 22: 9798 / 10000
Epoch 23: 9805 / 10000
Epoch 24: 9787 / 10000
Epoch 25: 9797 / 10000
Epoch 26: 9806 / 10000
Epoch 27: 9799 / 10000
Epoch 28: 9819 / 10000
Epoch 29: 9807 / 10000
Epoch 30: 9815 / 10000
Epoch 31: 9811 / 10000
Epoch 32: 9816 / 10000
Epoch 33: 9813 / 10000
Epoch 34: 9811 / 10000
Epoch 35: 9813 / 10000
Epoch 36: 9817 / 10000
Epoch 37: 9816 / 10000
Epoch 38: 9823 / 10000
Epoch 39: 9817 / 10000
Epoch 40: 9825 / 10000
Epoch 41: 9818 / 10000
Epoch 42: 9810 / 10000
Epoch 43: 9806 / 1000

In [8]:
model = {'min_batch':20, 'epoches':50, 'learningrate':3, 'comment': 'image_momentum_0.9'}
t1 = time.time()
training_data, validation_data, test_data = load_data_wrapper(f)
net = Network([784, 256,128,64, 10])
loss_Tr, loss_Te, acc_tr, acc_Te = net.SGD(training_data, model['epoches'] , model['min_batch'], model['learningrate'],model['comment'], test_data = test_data)

print(time.time() - t1)

Epoch 0: 1009 / 10000
Epoch 1: 1032 / 10000
Epoch 2: 1032 / 10000
Epoch 3: 1032 / 10000
Epoch 4: 1032 / 10000
Epoch 5: 1032 / 10000
Epoch 6: 1032 / 10000
Epoch 7: 1032 / 10000
Epoch 8: 1032 / 10000
Epoch 9: 1032 / 10000
Epoch 10: 1032 / 10000
Epoch 11: 1032 / 10000
Epoch 12: 1032 / 10000
Epoch 13: 1032 / 10000
Epoch 14: 1032 / 10000
Epoch 15: 1032 / 10000
Epoch 16: 1032 / 10000
Epoch 17: 1032 / 10000
Epoch 18: 1032 / 10000
Epoch 19: 1032 / 10000
Epoch 20: 1032 / 10000
Epoch 21: 1032 / 10000
Epoch 22: 1032 / 10000
Epoch 23: 1032 / 10000
Epoch 24: 1032 / 10000
Epoch 25: 1032 / 10000
Epoch 26: 1032 / 10000
Epoch 27: 1032 / 10000
Epoch 28: 1032 / 10000
Epoch 29: 1032 / 10000
Epoch 30: 1032 / 10000
Epoch 31: 1032 / 10000
Epoch 32: 1032 / 10000
Epoch 33: 980 / 10000
Epoch 34: 980 / 10000
Epoch 35: 980 / 10000
Epoch 36: 980 / 10000
Epoch 37: 980 / 10000
Epoch 38: 980 / 10000
Epoch 39: 980 / 10000
Epoch 40: 980 / 10000
Epoch 41: 980 / 10000
Epoch 42: 980 / 10000
Epoch 43: 980 / 10000
Epoch 44: